In [6]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "2NCD4p44jNbxqVXEFl5cEjcIg5fbqrNi5INs3VvPwQGPnt6zGWPKJQQJ99AJACmepeSXJ3w3AAABACOGAHUq"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://rag-chatapp.openai.azure.com/"

In [7]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4",
    api_version="2024-05-01-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
)

In [16]:
messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]
llm.invoke(messages)

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-6a771921-e6db-48b1-8b0f-b81391b46c2f-0', usage_metadata={'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'input_token_details': {}, 'output_token_details': {}})

In [17]:
for chunk in llm.stream(messages):
    print(chunk)

content='' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='J' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content="'" additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='ad' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='ore' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content=' programmer' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='.' additional_kwargs={} response_metadata={} id='run-93a9b3bf-f8a8-4eae-8cba-6f0ec478c637'
content='' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12'} id='run-93a9b3bf-f8a8-4eae-8cba-6f

In [18]:
stream = llm.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full

AIMessageChunk(content="J'adore programmer.", additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12'}, id='run-6926835d-fa2b-4ca2-8be3-06e7faeda91f')

In [19]:
await llm.ainvoke(messages)

AIMessage(content="J'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_5b26d85e12', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-b7aeb809-ed2a-4351-890d-b3cdb268fc71-0', usage_metadata={'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'input_token_details': {}, 'output_token_details': {}})

In [21]:
from typing import Optional

from pydantic import BaseModel, Field


class Joke(BaseModel):
    '''Joke to tell user.'''

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(description="How funny the joke is, from 1 to 10")


structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke about cats")

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7)

In [22]:
Joke(
    setup="Why was the cat sitting on the computer?",
    punchline="To keep an eye on the mouse!",
    rating=None,
)

Joke(setup='Why was the cat sitting on the computer?', punchline='To keep an eye on the mouse!', rating=None)

In [23]:
json_llm = llm.bind(response_format={"type": "json_object"})
ai_msg = json_llm.invoke(
    "Return a JSON object with key 'random_ints' and a value of 10 random ints in [0-99]"
)
ai_msg.content

'{\n  "random_ints": [45, 12, 87, 23, 56, 78, 34, 90, 62, 15]\n}'

In [4]:
from pydantic import BaseModel, Field


class GetWeather(BaseModel):
    '''Get the current weather in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


class GetPopulation(BaseModel):
    '''Get the current population in a given location'''

    location: str = Field(
        ..., description="The city and state, e.g. San Francisco, CA"
    )


llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])
ai_msg = llm_with_tools.invoke(
    "Which city is hotter today and which is bigger: LA or NY?"
)
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'call_xoqdXCzmPoavGZmtqtXy4IgZ',
  'type': 'tool_call'}]

In [3]:
import base64
import httpx
from langchain_core.messages import HumanMessage

image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the weather in this image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
)
ai_msg = llm.invoke([message])
ai_msg.content

RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-05-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}

In [5]:
ai_msg = llm.invoke(messages)
ai_msg.usage_metadata

NameError: name 'messages' is not defined

In [ ]:
logprobs_llm = llm.bind(logprobs=True)
ai_msg = logprobs_llm.invoke(messages)
ai_msg.response_metadata["logprobs"]